# Text Search Search in Korean


-   [소스 데이터](https://github.com/jbose038/naver-movie-recommendation/blob/master/dataset/movies04293.csv)


## 1. 필요한 패키지 설치


In [25]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

In [26]:
# Read data/movies.csv and save it in pandas dataframe
import os
import pandas as pd

movies_raw_df = pd.read_csv("./data/movies.csv", low_memory=False)
movies_raw_df.head(20)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url,keywords,plot_preprocessed_kmr,plot_preprocessed_kkma
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...,"['영국군', '중령', '대전', '한창', '공격', '병사', '부대', '수...",세계 대전 한창 독일군 통신망 파괴 상황 속 영국군 병사 딘 찰스 미션 함정 영국군...,세계 대전 한창 독일군 모든 통신망 파괴 상황 속 영국군 병사 채프먼 하나 미션 함...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...,"['연쇄', '범인', '잡기', '베테랑', '사건', '수단', '방법', '극...",뉴욕 맨해튼 중심 경찰 연쇄 살해 사건 범인 잡기 베테랑 경찰 극단 조치 시간 수단...,뉴욕 맨해튼 중심 경찰 연쇄 살해 사건 범인 베테랑 경찰 극단 조치 시간 수단 방법 놈
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...,"['사랑', '플로', '완벽', '가동', '후회', '천재', '충격', '물리...",여자친구 충격 물리학 천재 사랑 마음 두뇌 풀 가동 후회 순간 타임머신 을 개발 안...,여자 친구 차여 충격 물리학 천재 사랑 마음 두뇌 가동 후회 순간 타임머신 개발 때...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...,"['민주', '여행', '지연', '부부', '신혼', '제안', '사라', '예비...",결혼 예비 신혼 부부 차 민주 차 직장 후배 용 부부 커플 여행 제안 지연 불편 여...,결혼 예비 신혼 부부 차 과 민주 차 직장 후배 용 우 부부 커플 여행 제안 지연 ...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...,"['첫눈', '피아노', '부잣집', '프랑스', '아가씨', '마고', '서로',...",부잣집 프랑스 아가씨 마고 첫눈 발리 가난 어부 에카 피아노 개인 교습 부탁 건반 ...,부잣집 프랑스 아가씨 마 첫눈 발리 가난 어부 카 피아노 개인 교습 부탁 건반 사람...
5,갱,액션,2020,1.16,5.99,105,교도소를 개조해 만든 국내 최악의 고등학교 ‘대훈고’ 그리고 그 곳에 온 강제전학생...,차지혁|조선기|옥윤중|백재민|김대한|이정현,김라희|송은석|진성|이재한|맹재렬|이채성|최준민|시윤|지남혁|임정옥|권오윤|김동우|...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191115_141/1...,"['개조', '국내', '학생', '강제', '전학생', '목표', '배틀', '시...",교도소 개조 국내 최악 고등학교 곳 강제 전학생 최지훈 목표 현실 종소리 매점 대신...,교도소 개조 국내 최악 고등 학교 곳 강제 전 학생 지 훈 목표 단 이 현실 종 소...
6,건즈 아킴보,액션,2020,4.15,6.39,173,"낮엔 평범한 직장인, 밤엔 키보드 워리어 ‘마일즈’ 손가락 한번 잘못 놀렸다가 살인...",다니엘 래드클리프|사마라 위빙|네드 데네히|나타샤 류 보르디초,라이스 다비|재클린 리 거츠|콜린 모이,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200323_116/1...,"['게임', '시작', '구역', '플레이어', '설상가상', '관람', '익스트림...",낮 직장인 밤 키보드 워리어 손가락 살인 미션 레알 목숨 개념 게임 양손 권총 게임...,낮 직장인 밤 키보드 손가락 살인 미션 레 목숨 신 개념 게임 양손 권총 게임 강제...
7,걸즈 앤 판처 제 63회 전차도 전국 고교생 대회,애니메이션,2020,4.09,4.67,3,여성의 소양 중 하나로 손꼽히며 전통적인 무예의 교양 활동인 전차도로 유명한 니시즈...,후치가미 마이|카야노 아이|오자키 마미|이구치 유카,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200330_55/15...,"['전차', '참가', '전학', '도로', '교양', '활동', '폐교', '최종...",여성 소양 하나로 전통 무예 교양 활동 전차 도로 니 시즈 미 집안 전차 도가 아라...,여성 소양 전통적 무예 교양 활동 전차 도로 유명 시 미 집안 전차 라이 여 학원 ...
8,걸즈 앤 판처 최종장,애니메이션,2020,3.12,8.93,15,대학 선발팀과의 시합에서 우승하며 오아라이 여고를 폐교 위기에서 구해낸 미호와 전차...,후치가미 마이|카야노 아이|오자키 마미|이구치 유카|나카가미 이쿠미,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200303_117/1...,"['모모', '아라이', '여고', '우승', '경기', '전차', '자유', '고...",대학 선발팀 시합 우승 아라이 여고 폐교 위기 미 전차 팀원 선배 모모 유급 소식 ...,대학 선발팀 시합 우승 라이 여고 폐교 위기 미 호 전차 팀원 선배 모 모의 유급 ...
9,걸즈 앤 판처 최종장 제1화,애니메이션|드라마,2020,1.08,8.67,42,대학 선발팀과의 시합에서 우승하며 오아라이 여고를 폐교 위기에서 구해낸 미호와 전차...,후치가미 마이|카야노 아이|이구치 유카|오자키 마미|나카가미 이쿠미,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191231_269/1...,"['모모', '아라이', '여고', '우승', '경기', '전차', '자유', '고...",대학 선발팀 시합 우승 아라이 여고 폐교 위기 미 전차 팀원 선배 모모 유급 소식 ...,대학 선발팀 시합 우승 라이 여고 폐교 위기 미 호 전차 

In [100]:
# Remove unnecessary columns
movies_columns_removed_df = movies_raw_df.loc[:, "title":"img_url"]
movies_columns_removed_df.tail(1)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
11950,히 갓 게임,드라마,2000,3.25,7.69,45,현실 문제에서 예술 영화로 전환한 스파이크 감독의 작품. 흥행과 비평 모두 좋은 평...,덴젤 워싱턴|레이 앨런|밀라 요보비치|로사리오 도슨,힐 하퍼|젤다 해리스|네드 비티|짐 브라운|조셉 라일 테일러|빌 넌|미쉘 샤이|토머...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20111222_110/1...


In [131]:
# Nan 모두 제거
final_df = movies_columns_removed_df.fillna("정보없음")
final_df.to_csv("./data/movie_data.csv", index=False)

## OpenSearch에 데이터 인덱싱하기


### OpenSearch 클러스터에 연결


In [29]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

In [30]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region="us-east-1")

aos_host = "search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws"

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

### Index 생성

노리 플러그인을 사용하기 위해서는 Package를 Associate 해줘야 한다.


In [31]:
request_body = {"analyzer": "nori", "text": "이제 OpenSearch에서도 노리를 사용할 수 있습니다."}

# Send the request to the _analyze endpoint
response = aos_client.indices.analyze(body=request_body)

# Print the response
print(json.dumps(response, indent=4, ensure_ascii=False))

{
    "tokens": [
        {
            "token": "opensearch",
            "start_offset": 3,
            "end_offset": 13,
            "type": "word",
            "position": 1
        },
        {
            "token": "노리",
            "start_offset": 17,
            "end_offset": 19,
            "type": "word",
            "position": 4
        },
        {
            "token": "사용",
            "start_offset": 21,
            "end_offset": 23,
            "type": "word",
            "position": 6
        },
        {
            "token": "수",
            "start_offset": 25,
            "end_offset": 26,
            "type": "word",
            "position": 9
        },
        {
            "token": "있",
            "start_offset": 27,
            "end_offset": 28,
            "type": "word",
            "position": 10
        }
    ]
}


In [107]:
movies_default_index = {
    "settings": {
        "number_of_replicas": 2,
        "number_of_shards": 1,
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
    }
}

In [114]:
# aos_client.indices.delete(index="movies_default_index")
aos_client.indices.create(index="movies_default_index", body=movies_default_index)

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [movies_default_index/7pUDiFw2TFaNRXDdReQQOQ] already exists')

In [115]:
aos_client.indices.get(index="movies_default_index")

{'movies_default_index': {'aliases': {},
  'mappings': {},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '5',
    'provided_name': 'movies_default_index',
    'creation_date': '1711607506572',
    'number_of_replicas': '2',
    'uuid': '7pUDiFw2TFaNRXDdReQQOQ',
    'version': {'created': '136327827'}}}}}

In [120]:
response = aos_client.delete_by_query(
    index="movies_default_index", body={"query": {"match_all": {}}}
)

In [121]:
from tqdm import tqdm
from opensearchpy import helpers

# Ingest the DataFrame
# for idx, row in tqdm(final_df.iterrows()):
# aos_client.index(index="movies_default_index", body=row.to_dict(), id=idx)

json_data = final_df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": "movies_default_index", "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(aos_client, actions=_generate_data()):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

# Refresh the index to make the changes visible
aos_client.indices.refresh(index="movies_default_index")

{'_shards': {'total': 15, 'successful': 15, 'failed': 0}}

In [122]:
res = aos_client.search(index="movies_default_index", body={"query": {"match_all": {}}})
print("Records found: %d " % res["hits"]["total"]["value"])
count = aos_client.count(index="movies_default_index")
print(count)

Records found: 10000 
{'count': 11951, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


## 기본적인 Text Search


In [129]:
query = {
    "size": 30,
    "query": {
        "multi_match": {
            "query": "영웅들이 모여서 악당을 해치운다",
            "fields": ["title", "plot", "genre"],
        }
    },
}
res = aos_client.search(index="movies_default_index", body=query)
query_result = []
for hit in res["hits"]["hits"]:
    row = [
        hit["_id"],
        hit["_score"],
        hit["_source"]["title"],
        hit["_source"]["plot"],
        hit["_source"]["genre"],
        hit["_source"]["rating"],
    ]
    query_result.append(row)

query_result_df = pd.DataFrame(
    data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
)
display(query_result_df)

,_id,_score,title,plot,genre,rating
0,qCzTg44BnEOV7ezVNRRt,10.075981,협객,자신의 이름을 버리고 사라진 영웅 ‘시진’ 그와 함께 대의를 꿈꾸던 영웅들이 남긴 ...,액션,4.57
1,4xTTg44B5vU2gwgcPLQG,9.369944,스타워즈 반란군,오직 어둠만이 존재하는 은하계 제국의 통치 아래... 빛의 포스로 맞서 싸우는 용감...,애니메이션,9.50
2,HizTg44BnEOV7ezVNRNt,9.258982,샤잠!,"솔로몬의 지혜, 헤라클레스의 힘, 아틀라스의 체력, 제우스의 권능, 아킬레스의 용기...",액션|판타지|SF,5.82
3,KxTTg44B5vU2gwgcRNDr,8.935827,잭애스,스턴트의 달인 아홉 명이 모여서 각종 엽기적인 스턴트를 선보인다. 일반인들은 도저히...,다큐멘터리|액션|코미디,7.49
4,nyzTg44BnEOV7ezVNRWj,8.863059,마중: 커피숍 난동 수다 사건,어렸을 때부터 동거동락한 7명의 친구들 이제는 세월을 흘러서 양복이 더 잘 어울리는...,코미디|드라마,6.37
5,PizTg44BnEOV7ezVNRIu,8.505411,닌자터틀 : 어둠의 히어로,감옥을 탈출한 ‘슈레더’는 인간을 동물로 바꿀 수 있는 의문의 보라색 액체를 손에 ...,액션|모험|코미디|판타지|SF,8.35
6,qSzTg44BnEOV7ezVNRWj,8.332884,매드 몰리,"전쟁으로 멸망한 미래, 이제 식량과 약품을 가진 자가 살아남은 자들 위에 선다. 생...",액션,3.67
7,ASzTg44BnEOV7ezVQS1Y,8.291601,가필드 펫 포스 3D,"먹는 게 취미고, 잠자는 게 특기인 게으른 고양이 가필드! 걱정고민 없이 ‘카툰월드...",애니메이션|코미디|가족,6.36
8,YyzTg44BnEOV7ezVNRIu,7.995084,데몬킹스,"무적의 고수, 데몬킹의 전설이 전해오는 11세기 중국. 호시탐탐 왕위를 노리던 악당...",애니메이션|액션|모험,5.50
9,-BTTg44B5vU2gwgcQMCZ,7.980578,킥 오프,전쟁으로 폐허가 된 이라크. 폭탄 테러가 일상이 돼버린 도시 키르쿠크의 파손된 스타...,드라마|전쟁,7.51
